In [5]:
import subprocess
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tempfile

In [6]:
blender_path = "E:\programs\\blender\\blender-4.0\\blender.exe"
# First create a temporary Python script that Blender will run
blender_script = """
import bpy
import numpy as np

def reset_scene():
    # Clear existing objects
    bpy.ops.object.select_all(action='SELECT')
    bpy.ops.object.delete()
    
    # Create camera
    bpy.ops.object.camera_add(location=(4, -4, 3))
    camera = bpy.context.active_object
    camera.rotation_euler = (3.14159/4, 0, 3.14159/4)
    
    # Create light
    bpy.ops.object.light_add(type='SUN', location=(5, 5, 5))
    
    return camera

def setup_camera(camera):
    camera.data.lens = 35
    camera.data.sensor_width = 32
    camera.data.sensor_height = 18
    camera.data.clip_start = 0.1
    camera.data.clip_end = 100
    return camera

def create_box(location, size=1.0, color=(1,0,0,1)):
    bpy.ops.mesh.primitive_cube_add(size=size, location=location)
    box = bpy.context.active_object
    
    # Create material
    material = bpy.data.materials.new(name="BoxMaterial")
    material.use_nodes = True
    material.node_tree.nodes["Principled BSDF"].inputs[0].default_value = color
    
    if box.data.materials:
        box.data.materials[0] = material
    else:
        box.data.materials.append(material)
    
    return box

def main():
    # Reset and setup scene
    camera = reset_scene()
    setup_camera(camera)
    
    # Create boxes
    boxes = [
        create_box((-1, -1, 0.5), size=1.0, color=(1, 0, 0, 1)),
        create_box((1, 1, 0.5), size=1.0, color=(0, 1, 0, 1)),
        create_box((0, 0, 1.5), size=0.8, color=(0, 0, 1, 1))
    ]
    
    # Set render properties
    bpy.context.scene.render.engine = 'CYCLES'
    bpy.context.scene.render.resolution_x = 256
    bpy.context.scene.render.resolution_y = 256
    bpy.context.scene.render.image_settings.file_format = 'PNG'
    bpy.context.scene.render.filepath = r'TEMP_PATH_PLACEHOLDER'
    
    # Render
    bpy.ops.render.render(write_still=True)

if __name__ == "__main__":
    main()
"""

def render_blender_scene():
    # Create a temporary directory for our files
    with tempfile.TemporaryDirectory() as temp_dir:
        # Create paths for our temporary files
        script_path = os.path.join(temp_dir, 'blender_script.py')
        output_path = os.path.join(temp_dir, 'render_output.png')
        
        # Replace the placeholder path in our script with the actual output path
        blender_script_with_path = blender_script.replace('TEMP_PATH_PLACEHOLDER', output_path)
        
        # Write the Blender script to a temporary file
        with open(script_path, 'w') as f:
            f.write(blender_script_with_path)
        
        # Run Blender with our script
        # Note: Replace 'blender' with the full path to your Blender executable if needed
        try:
            result = subprocess.run(
                [blender_path, '--background', '--python', script_path],
                capture_output=True,
                text=True
            )
            
            if result.returncode != 0:
                print("Blender Error:")
                print(result.stderr)
                return None
            
            # Load the rendered image
            img = np.array(Image.open(output_path))
            return img

        except FileNotFoundError:
            print("Error: Blender executable not found!")
            print("Please make sure Blender is installed and in your system PATH")
            print("Or replace 'blender' with the full path to your Blender executable")
            return None

# Render the scene and display the result
render_array = render_blender_scene()

if render_array is not None:
    # Display the image
    plt.figure(figsize=(8, 8))
    plt.imshow(render_array)
    plt.axis('off')
    plt.show()
    
    # Print array information
    print(f"Render array shape: {render_array.shape}")
    print(f"Value range: [{render_array.min()}, {render_array.max()}]")

Blender Error:
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "C:\Users\tim-s\AppData\Local\Temp\tmpfqp9r9_v\blender_script.py", line 67, in <module>
    main()
  File "C:\Users\tim-s\AppData\Local\Temp\tmpfqp9r9_v\blender_script.py", line 64, in main
    bpy.ops.render.render(write_still=True)
  File "E:\programs\blender\blender-4.0\4.0\scripts\modules\bpy\ops.py", line 109, in __call__
    ret = _op_call(self.idname_py(), kw)
RuntimeError: Error: Cannot render, no camera


